In [2]:
import sys
sys.path.append('..')

In [3]:
from pathlib import Path

In [31]:
import cv2
import numpy as np
import pandas as pd

In [5]:
from lib.evaluate import Evaluator

In [167]:
images = sorted(Path('/home/jiun/datasets/amano/nvr-warp/test/images').glob('*.jpg'))
gts = sorted(Path('/home/jiun/datasets/amano/nvr-warp/test/annotations').glob('*.txt'))
dts = sorted(Path('/home/jiun/datasets/amano/nvr-warp/results/pretrained').glob('*.txt'))
rts = sorted(Path('/home/jiun/datasets/amano/nvr-warp/results/pretrained-reversed').glob('*.txt'))

In [168]:
assert len(gts) == len(dts) == len(rts)

In [169]:
evaluator = Evaluator(num_classes=2)
counts = 0
gcounts = 0

In [170]:
for index, (gt, dt, rt) in enumerate(zip(gts, dts, rts)):
    g = pd.read_csv(str(gt), header=None).values
    d = pd.read_csv(str(dt), header=None).values
    r = pd.read_csv(str(rt), header=None).values
    
    target_mask = d[:, 0] == 7
    conf_mask = d[:, 1] > .3
    dd = d[target_mask & conf_mask]

    target_mask = r[:, 0] == 7
    conf_mask = r[:, 1] > .3
    rr = r[target_mask & conf_mask]
    
    rr[:, 3::2] = 1080 - rr[:, 3::2]
    
    boxes = np.concatenate((dd, rr))
    
    counts += np.size(boxes, 0)
    gcounts += np.size(g, 0)
    
#     image = cv2.imread(str(images[index]))
    
#     for x, y, w, h in boxes[:, 2:]:
#         cv2.rectangle(image, (int(x), int(y)), (int(w), int(h)), (255, 0, 0), 1)
        
#     for x, y, w, h in g:
#         cv2.rectangle(image, (int(x), int(y)), (int(w), int(h)), (0, 255, 0), 1)
    
#     show(image)
    
    evaluator.update((
        np.ones(np.size(boxes, 0), dtype=np.int),
        boxes[:, 1].astype(np.float32),
        boxes[:, 2:].astype(np.float32),
        None,
    ), (
        np.ones(np.size(g, 0), dtype=np.int),
        g.astype(np.float32),
        None,
    ))

In [171]:
# nvr-warp
print(evaluator.mAP)
print(counts)

[0.04345392 0.04345392]
9485


In [161]:
# nvr-warp-2pi
print(evaluator.mAP)
print(counts)

[0.00052445 0.00052445]
6582


In [166]:
# nvr-warp-4pi
print(evaluator.mAP)
print(counts)

[0.00793071 0.00793071]
5807


In [33]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))